## Part 3: data acquisition and processing of rent data from the Quandl API

Finally, let's take a look at the rental prices across LA. For the previous section we subdivided LA using the LA Times neighborhood polygons. Unfortunately, I couldn't find rental data for these specific neighborhood divisions. Instead, we'll use the zipcode-defined regions found on the <a href="https://data.lacounty.gov/Geospatial/ZIP-Codes/65v5-jw9f"> County of Los Angeles Open Data site </a> in GeoJSON format. Then, the median rental price per square foot for each zipcode is accessed from Zillow via Quandl.  

In [1]:
import quandl
import pandas as pd
import folium
from shapely.geometry import Point
import geopandas as gpd
import haversine
quandl.read_key()

First we cut down the dataset to regions within a given radius of Downtown. We'll make the radius slightly bigger (35 km), so the clusters in the next section have some overlap with the restaurant dataset.

In [2]:
la_geo = r'raw/ZIP_Codes.geojson'
zip_polys = gpd.read_file(la_geo)
zip_polys['centroid'] = zip_polys.centroid
zip_polys.drop(['shape_area','shape_len','objectid'],inplace=True,axis=1)
centroids = gpd.read_file('processed/la_times_centroids.shp')
centroids.columns = ['name', 'r_list', 'centroids']
centroids.set_index("name",inplace=True)

In [3]:
distances_list = haversine.haver([a.x for a in zip_polys['centroid']],
                                 [a.y for a in zip_polys['centroid']],
                                 centroids.loc['Downtown']['centroids'].x,
                                 centroids.loc['Downtown']['centroids'].y)
zip_polys['distance_to_DTLA'] = distances_list#Find distance to DTLA for each zipcode centroid
zip_polys.head(5)

,zipcode,geometry,centroid,distance_to_DTLA
0,90001,(POLYGON ((-118.2433683530738 33.9892312651088...,POINT (-118.2494118600075 33.97367691867435),7.273024
1,90002,(POLYGON ((-118.2342946301028 33.9610050734310...,POINT (-118.2470381405668 33.94909020305977),10.000298
2,90003,(POLYGON ((-118.2739239050975 33.9893399471187...,POINT (-118.2739369067153 33.96410280345462),8.722033
3,90004,(POLYGON ((-118.2840918468618 34.0834829264284...,POINT (-118.310938222861 34.07620349892411),7.279293
4,90005,(POLYGON ((-118.3365046094729 34.0618919193627...,POINT (-118.3100257229236 34.05979472135997),6.343185


In [4]:
zip_polys2 = gpd.GeoDataFrame(zip_polys[zip_polys['distance_to_DTLA'] < 35]) #apply filter at 35 km radius
zip_polys2.drop('centroid',inplace=True,axis=1)

Now that we have the zipcodes and associated polygons within 35 km of DTLA, we define a function to call Quandl for rental data. The keyword 'ZRIFAH' in the following code corresponds to the estimated median rental price per square foot as calculated by Zillow. This query returns data in a month-by-month format, so we just find the average of the last 12 months. One interesting followup would be to check the rental growth rate in each neighborhood using the temporal data, but we'll leave that out for now.

In [5]:
def get_rent_prices(zipcode_list):
    rent_list = []
    for zipcode in zipcode_list:
        try: 
            rent_df = quandl.get("ZILLOW/Z{}_ZRIFAH".format(str(zipcode)))
            last_years_rent = rent_df.tail(12).mean() #return mean rent for last 12 months
            rent_list.append((zipcode,float(last_years_rent)))
        except:
            rent_list.append((zipcode,None))
        print(zipcode)
    return(pd.DataFrame(rent_list,columns=['zipcode','rent_psf']))

Get rent data for all zipcodes in the zip_polys2 dataframe and save to file:

In [6]:
prices_list = get_rent_prices(zip_polys2['zipcode'])
prices_list

90001
90002
90003
90004
90005
90006
90007
90008
90010
90011
90012
90013
90014
90015
90016
90017
90018
90019
90020
90021
90022
90023
90024
90025
90026
90027
90028
90029
90031
90032
90033
90034
90035
90036
90037
90038
90039
90040
90041
90042
90043
90044
90045
90046
90047
90048
90049
90056
90057
90058
90059
90061
90062
90063
90064
90065
90066
90071
90831
90067
90068
90069
90089
90094
90073
90077
90241
90242
90095
90201
90210
91046
91101
91103
91104
91105
90211
91106
91107
91108
90212
90220
90221
90222
90230
90232
90240
90245
90247
90248
90249
90250
91345
91352
91356
90254
90255
90260
90261
90262
90266
90270
90272
90274
90275
90277
90278
90280
90290
90291
90292
90293
90301
90302
90303
90304
90305
90401
90402
90403
90404
90405
90501
90502
90503
90504
90505
90506
90601
90602
90603
90604
90605
90606
90623
90630
90631
90638
90747
90755
90802
90803
90804
90805
90806
90639
90640
90650
90660
90670
90701
90703
90706
90710
90712
90807
90808
90810
90813
90814
90815
90713
90715
90716
90717
90723
9073

,zipcode,rent_psf
0,90001,2.381667
1,90002,2.276333
2,90003,2.344833
3,90004,2.743500
4,90005,2.250500
5,90006,2.249667
6,90007,1.888500
7,90008,1.955333
8,90010,2.573167
9,90011,2.168833


Join with the zip_polys:

In [7]:
prices_list.dropna(inplace=True)
rent_polys_df = gpd.GeoDataFrame(prices_list.merge(zip_polys2,how='inner',left_on='zipcode',right_on='zipcode'))

Save to file

In [8]:
rent_polys_df.to_file("processed/rent_data.shp")

C:\Users\ianfi\Anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [ ]:
# This allows NB viewer to use scrolling on the long outputs
%%html
<style>
.nbviewer div.output_area {
  overflow-y: auto;
  max-height: 500px; /* or value of your choosing */
}
</style>